In [ ]:
import pandas as pd
import altair as alt
import vega_datasets
alt.data_transformers.disable_max_rows()

In [4]:
hatecrimes = pd.read_csv("../data/hate_crime.csv")

In [ ]:
alt.Chart(hatecrimes, title="Seasonality of Hate Crimes").mark_rect().encode(
    alt.X('date(incident_date):O').title("Date"),
    alt.Y('month(incident_date):O').title("Month"),
    alt.Color('count()').title("Incidents").scale(scheme='orangered')
).display()

In [ ]:
alt.Chart(hatecrimes, title='Hate Crimes Over Time').mark_line().encode(
        alt.X('data_year:O').title("Year"),
        alt.Y('count():N').title("Hate Crime Incidents"),
        color=alt.value("#ad2e03")
    ).display()

In [ ]:
alt.Chart(hatecrimes).mark_line().encode(
        x='data_year:O',
        y = 'count():N',
        color='bias_desc:N'
    ).display()

In [ ]:
# Cleaning pandas 'bias description' variable
#hatecrimes.groupby(['bias_desc']).size().sort_values(ascending=False).head(10)
# Most popular hate crimes are anti Black, Jewish, White, Gay, Hispanic, Other Race, LGBT, Asian, Multi-Race, Lesbian

In [ ]:
top10 = ["Anti-Black or African American", "Anti-Jewish", "Anti-White", "Anti-Gay (Male)", "Anti-Hispanic or Latino", "Anti-Other Race/Ethnicity/Ancestry","Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)", "Anti-Asian", "Anti-Multiple Races, Group", "Anti-Lesbian (Female)"]
hatecrimestop10 = hatecrimes.loc[hatecrimes['bias_desc'].isin(top10)]
alt.Chart(hatecrimestop10).mark_line().encode(
        alt.X('data_year:O').title("Year"),
        alt.Y('count():N').title("Incidents"),
        alt.Color('bias_desc:N').title("Crime Bias").scale(scheme='category10')
    ).display()

In [ ]:
# Here we can see that Anti-Black hate crimes are far more prevalent than other hate crimes, but have decreased from 2022-2023. Anti-Jewish hate crimes on the other hand, have risen since 2021

In [6]:
from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, feature='states')

In [7]:
pop = data.population_engineers_hurricanes(usecols=['state', 'id'])

In [ ]:
hatecrimes_w_ids = hatecrimes.merge(pop, left_on='state_name', right_on='state')
state_counts = hatecrimes_w_ids.groupby(['id'], as_index=False).size()


In [ ]:
states = alt.topo_feature(data.us_10m.url, 'states')
alt.Chart(states, title="Hate Crime Map").mark_geoshape().encode(
    alt.Color('size:Q').scale(scheme='browns').title("Hate Crime Incidents")
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_counts, 'id', list(state_counts.columns))
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

In [ ]:
# Total heatmap of all hatecrimes in continental US -- consider adjusting for population

In [ ]:
hatecrimes.groupby(['offense_name']).size().sort_values(ascending=False).head(10)

In [ ]:
alt.Chart(hatecrimes).mark_bar().encode(
        x='offender_race:O',
        y = 'count():N'
    ).display()

In [4]:
# Tracking hate crimes specifically in 2019-2023
years = [2019, 2020, 2021, 2022, 2023]
hatecrimes1924= hatecrimes.loc[hatecrimes['data_year'].isin(years)]

In [7]:
hatecrimes1924.groupby(['bias_desc']).size().sort_values(ascending=False).head(10)
top10_rec = ["Anti-Black or African American", "Anti-Jewish", "Anti-White", "Anti-Gay (Male)", "Anti-Hispanic or Latino", "Anti-Other Race/Ethnicity/Ancestry","Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)", "Anti-Asian", "Anti-Multiple Races, Group", "Anti-Transgender"]
hatecrimestop10_rec = hatecrimes1924.loc[hatecrimes1924['bias_desc'].isin(top10_rec)]


In [ ]:
main_c = alt.Chart(hatecrimestop10_rec, title="Top Hate Crimes From 2019-2023").mark_line().encode(
        alt.X('yearmonth(incident_date):O').title('Month'),
        alt.Y('count()').title('Hate Crime Incidents'), 
        color='bias_desc:N'
    )

text1 = alt.Chart({'values': [{'x': 100, 'y': 500}]}).mark_text(
    text="Murder of George Floyd").encode(
        alt.X('x:Q', axis=None),
        y='y:Q')

text2 = alt.Chart({'values': [{'x': 260, 'y': 150}]}).mark_text(
    text="2021 Atlanta Spa Shootings").encode(
        alt.X('x:Q', axis=None),
        y='y:Q')

text3 = alt.Chart({'values': [{'x': 640, 'y': 300}]}).mark_text(
    text="Hamas Attacks Israel").encode(
        alt.X('x:Q', axis=None).scale(domain=(-100, 100)),
        y='y:Q')

text = text3 + text2 + text1
main_c + text
# Want to add text here to note particular events that saw a spike -- George Floyd's murder, the 2021 Atlanta spa shootings, Oct. 7